<a href="https://colab.research.google.com/github/sv4u/hashtag-generator/blob/master/HastagGeneratorDev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hashtag Generator Development

__This program takes any user-selected hashtag as input and outputs a list of the top 30 hashtags optimal for increasing a post's engagement.__

# User Guide

__Input your hashtag into the first argument of the getHashtagInfo() function in the main program down below. Run the code cell to view the table of results.__

## Import Statements

TODO: insert reasoning behind each import

In [1]:
!pip install git+https://github.com/ace-it-dev-interns/hashtag-generator.git

  Running command git clone -q https://github.com/ace-it-dev-interns/hashtag-generator.git 'C:\Users\bella\AppData\Local\Temp\pip-req-build-9rn95c33'
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\bella\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\bella\\AppData\\Local\\Temp\\pip-req-build-9rn95c33\\setup.py'"'"'; __file__='"'"'C:\\Users\\bella\\AppData\\Local\\Temp\\pip-req-build-9rn95c33\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\bella\AppData\Local\Temp\pip-pip-egg-info-ie135vuj'
         cwd: C:\Users\bella\AppData\Local\Temp\pip-req-build-9rn95c33\
    Complete output (5 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\bella\anaconda3\lib\tokenize.py", line 392, in open
        buffer = _


  Cloning https://github.com/ace-it-dev-interns/hashtag-generator.git to c:\users\bella\appdata\local\temp\pip-req-build-9rn95c33


In [2]:
import datetime
import requests
import json
import insta_data

# Main Program

__Run this cell after inputting the hashtag name.__

In [ ]:
params = insta_data.getCreds()
hashtag_info = insta_data.getHashtagInfo( 'blogger', params ) #enter hashtag name here
hashtag_id = hashtag_info['json_data']['data'][0]['id']
top_response, recent_response = insta_data.getHashtagMediaAll(hashtag_id, params)
comment_counts = insta_data.getMediaCommentCount(top_response, recent_response)
like_counts = insta_data.getMediaLikeCount(top_response, recent_response)
captions_and_ids = insta_data.getMediaCaptions(top_response, recent_response)
related_hashtags = insta_data.getRelatedHashtags(captions_and_ids)
list_of_dictionaries = insta_data.compilePostData(like_counts, comment_counts, captions_and_ids, related_hashtags)
number_of_posts = insta_data.getNumberOfPosts(related_hashtags, list_of_dictionaries)
total_likes = insta_data.getLikeTotals(related_hashtags, list_of_dictionaries)
total_comments = insta_data.getCommentTotals(related_hashtags, list_of_dictionaries)
hashtag_data = insta_data.compileHashtagData(related_hashtags, list_of_dictionaries, number_of_posts, total_likes, total_comments)
df = pd.DataFrame(hashtag_data).T.sort_values('total_comments')
df['avg_engagement'] = (df['total_likes'] + df['total_comments']) / df['number_of_posts']
df.sort_values('number_of_posts', ascending=False).head(30)